## Biomedical QA Dataset (Entity-Based)

This notebook creates a weakly supervised biomedical QA dataset using:
- Abstracts from PubMed
- Extracted biomedical entities (via SciSpaCy)

Each QA entry includes:
- A question generated from a biomedical entity
- A context (the abstract)
- An answer (initially the full abstract as a placeholder)

The output is saved in JSONL format and can be used for training biomedical question answering models (T5, BART, BioGPT, etc.).

> Future enhancements:
> - Replace answer with entity-specific span from the context
> - Use a retriever (e.g. keyword/entity search) to enhance answer grounding


In [ ]:
!pip install jsonlines

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Imports
import json
from tqdm import tqdm
import random
import jsonlines


In [ ]:
# Load enriched abstracts with biomedical entities
input_path = "/content/drive/MyDrive/biomedical_text_generation/data/enriched/abstracts_with_entities.json"

with open(input_path, "r", encoding="utf-8") as f:
    abstracts = json.load(f)

print(f"Loaded {len(abstracts)} abstracts.")


In [ ]:
# Question templates for entity-based QA
QUESTION_TEMPLATES = [
    "What is the role of {}?",
    "How does {} affect cancer?",
    "What do we know about {}?",
    "What is {}?",
    "How is {} used in treatment?"
]


In [ ]:
# Generate QA-style entries using biomedical entities
qa_data = []

for entry in tqdm(abstracts):
    pmid = entry.get("pmid")
    abstract = entry["abstract"]
    entities = entry.get("entities", [])

    for entity in entities:
        if len(entity.split()) < 2:
            continue  # Skip overly generic entities

        question = random.choice(QUESTION_TEMPLATES).format(entity)

        qa_data.append({
            "pmid": pmid,
            "context": abstract,
            "question": question,
            "answer": abstract  # Weak supervision: using full abstract as answer
        })


In [ ]:
import os

# Ensure the target directory exists
output_dir = "/content/drive/MyDrive/biomedical_text_generation/data/training/QA"
os.makedirs(output_dir, exist_ok=True)

# Save dataset to JSONL file
output_path = os.path.join(output_dir, "qa_dataset.jsonl")

with jsonlines.open(output_path, mode="w") as writer:
    writer.write_all(qa_data)

print(f"Saved {len(qa_data)} QA pairs to:")
print(output_path)
